In [ ]:
"""
Assignment 1 - music genre classification
"""


import pdb
import numpy as np
import pandas as pd
import os
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import validation_curve, GridSearchCV, StratifiedKFold, train_test_split
from xgboost import XGBClassifier
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import EarlyStopping

!pip install scikeras
from scikeras.wrappers import KerasClassifier
!pip install tensorflow-gpu
!nvidia-smi


In [ ]:
train_df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/data/gtzan_music_genre/features_30_sec.csv')
train_df.head()
print(train_df.shape)
# print(train_df.info())

(1000, 60)


In [ ]:
# Preprocessing
train_df = train_df.drop(['filename', 'length'], axis = 1)
y = train_df['label']
X = train_df.drop('label', axis = 1)
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
scale = MinMaxScaler()
scaled_data = scale.fit_transform(x_train)
x_train_sc = pd.DataFrame(scaled_data, columns = x_train.columns).values
scaled_data = scale.fit_transform(x_test)
x_test_sc = pd.DataFrame(scaled_data, columns = x_test.columns).values

In [ ]:
# Define a function to create the Keras model

def create_model2(layers, optim):
    model= Sequential()
    for i, nodes in enumerate(layers):
        if i==0:
            model.add(Dense(nodes, input_dim=x_train_sc.shape[1]))
        else:
            model.add(Dense(nodes, activation="relu"))
    model.add(Dense(len(y_train.unique()), activation='softmax'))

    model.compile(optimizer=optim, loss='binary_crossentropy', metrics=['accuracy'])
    return model

# need to set the input dim to input params and output dim to possible classes
def create_model(optimizer='adam', activation='relu', neurons=16):
    model = Sequential()
    model.add(Dense(neurons, input_dim=x_train_sc.shape[1], activation=activation))
    # model.add(Dense(y_train.shape[1], activation='softmax'))
    model.add(Dense(len(y_train.unique()), activation='softmax'))
    model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

# # Wrap Keras model into a function for compatibility with GridSearchCV
keras_model = KerasClassifier(build_fn=create_model, activation='relu', neurons=8, epochs=20)
# # Define the hyperparameter grid
param_grid = {
    'optimizer': ['adam', 'sgd', 'rmsprop'],
    'activation': ['relu', 'sigmoid', 'tanh'],
    'neurons': [8, 16, 32]
}
# # Create the GridSearchCV object
grid = GridSearchCV(estimator=keras_model, param_grid=param_grid, cv=5,
                    scoring='accuracy', verbose=9, n_jobs=-1)
# # Fit the model to the data
grid_result = grid.fit(x_train_sc, y_train, callbacks=[EarlyStopping(monitor='val_loss', patience=2)])

# model2= KerasClassifier(model=create_model2, verbose=0, optim='adam', layers=20)
# layers=[[20],[40,20], [45, 30, 15]]
# # activations = ['sigmoid','relu', 'tanh']
# optimizers = ['adam', 'sgd', 'rmsprop'],
# param_grid = dict(layers=layers, optim=optimizers, epochs=[30])
# grid = GridSearchCV(estimator=model2, param_grid=param_grid)

# grid_result= grid.fit(x_train_sc, y_train)


[grid_result.best_score_,grid_result.best_params_]
# Print the best hyperparameters
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

# Evaluate the model on the test set with the best hyperparameters
best_model = grid_result.best_estimator_
test_accuracy = best_model.score(x_test_sc, y_test)
print("Test Accuracy: %.2f%%" % (test_accuracy * 100))

In [ ]:
import tensorflow as tf
print("GPU Available:", tf.config.list_physical_devices('GPU'))

GPU Available: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [ ]:
# Now doing MNIST

# get data
DATA_PATH = '/content/drive/My Drive/Colab Notebooks/data/mnist/mnist_train.csv'
train_df = pd.read_csv(DATA_PATH)
DATA_PATH = '/content/drive/My Drive/Colab Notebooks/data/mnist/mnist_test.csv'
test_df = pd.read_csv(DATA_PATH)
print(train_df.shape)


(60000, 785)


In [ ]:
# need to set the input dim to input params and output dim to possible classes
def create_model(optimizer='adam', activation='relu', neurons=16):
    model = Sequential()
    model.add(Dense(neurons, input_dim=x_train_sc.shape[1], activation=activation))
    # model.add(Dense(y_train.shape[1], activation='softmax'))
    model.add(Dense(len(y_train.unique()), activation='softmax'))
    model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

# # Wrap Keras model into a function for compatibility with GridSearchCV
keras_model = KerasClassifier(build_fn=create_model, activation='relu', neurons=8, epochs=20)
# # Define the hyperparameter grid
param_grid = {
    'optimizer': ['adam', 'sgd', 'rmsprop'],
    'activation': ['relu', 'sigmoid', 'tanh'],
    'neurons': [8, 16, 32]
}
# # Create the GridSearchCV object
grid = GridSearchCV(estimator=keras_model, param_grid=param_grid, cv=5,
                    scoring='accuracy', verbose=9, n_jobs=-1)
# # Fit the model to the data
grid_result = grid.fit(x_train_sc, y_train, callbacks=[EarlyStopping(monitor='val_loss', patience=2)])

[grid_result.best_score_,grid_result.best_params_]
# Print the best hyperparameters
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

# Evaluate the model on the test set with the best hyperparameters
best_model = grid_result.best_estimator_
test_accuracy = best_model.score(x_test_sc, y_test)
print("Test Accuracy: %.2f%%" % (test_accuracy * 100))


Fitting 5 folds for each of 27 candidates, totalling 135 fits
Epoch 1/20
25/25 [==============================] - 2s 4ms/step - loss: 2.3384 - accuracy: 0.1437
Epoch 2/20
25/25 [==============================] - 0s 3ms/step - loss: 2.2336 - accuracy: 0.2050
Epoch 3/20
25/25 [==============================] - 0s 3ms/step - loss: 2.1557 - accuracy: 0.2763
Epoch 4/20
25/25 [==============================] - 0s 4ms/step - loss: 2.0761 - accuracy: 0.3462
Epoch 5/20
25/25 [==============================] - 0s 4ms/step - loss: 1.9932 - accuracy: 0.4075
Epoch 6/20
25/25 [==============================] - 0s 4ms/step - loss: 1.9152 - accuracy: 0.4137
Epoch 7/20
25/25 [==============================] - 0s 4ms/step - loss: 1.8377 - accuracy: 0.4450
Epoch 8/20
25/25 [==============================] - 0s 4ms/step - loss: 1.7639 - accuracy: 0.4688
Epoch 9/20
25/25 [==============================] - 0s 4ms/step - loss: 1.6999 - accuracy: 0.4575
Epoch 10/20
25/25 [==============================] - 0s 